# 6. Target Encoding

## 설명
- categorical features를 encode할 때 `target`을 사용한다.
- High-cardinality features에 적용하기 좋다.
- mean encoding 예시
    - 각 카테고리별 target의 평균으로 encoding 진행

    <img src="assets/target_encoding.jpg" width="50%"/>

- target encoding의 문제점
    1. `unknown categories`: target encoding은 overfitting의 위험이 있기 때문에 독립된 encoding split에서 학습되어야 하는데, encoding을 학습 데이터에 적용할 때 만약 encoding split에서 나타나지 않은 category가 있을 경우 결측값 처리를 해주어야 한다.
    2. `rare categories`: 데이터셋에서 드물게 나타난 category의 경우 encoding 결과가 해당 category를 대표한다고 볼 수 없다.

- 해결책: smoothing
    - `in-category` 평균과 `overall` 평균을 섞는 방법
    ``` python
    weight = n / (n + m)

    encoding = weight * in_category + (1 - weight) * overall
    ```
    - rare category의 weight은 작고, missing category는 overall average만 적용된다.
    - n: 데이터셋에서 해당 category가 나타난 총 횟수
    - m: smoothing factor를 결정한다. m이 크면 overall estimate에 더 큰 가중치를 부여한다.

## 코드

In [ ]:
X = df.copy()
y = X.pop('Rating')

# encoding split
X_encode = X.sample(frac=0.25)
y_encode = y[X_encode.index]

X_pretrain = X.drop(X_encode.index)
y_train = y[X_pretrain.index]

In [ ]:
from category_encoders import MEstimateEncoder

# Create the encoder instance. Choose m to control noise.
encoder = MEstimateEncoder(cols=["Zipcode"], m=5.0)

# Fit the encoder on the encoding split.
encoder.fit(X_encode, y_encode)

# Encode the Zipcode column to create the final training data
X_train = encoder.transform(X_pretrain)